In [2]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import median, mean
from collections import Counter

In [3]:
lr = 1e-3
env = gym.make("CartPole-v0")
env.mode = 'human'
env.reset()
goal_steps = 500
score_req = 100
initial_games = 60000

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [4]:
if not hasattr(env.action_space, 'n'):
    raise Exception('Keyboard agent only supports discrete action spaces')
ACTIONS = env.action_space.n
SKIP_CONTROL = 0    # Use previous control decision SKIP_CONTROL times, that's how you
                    # can test what skip is still usable.

human_agent_action = 0
human_wants_restart = False
human_sets_pause = False

def key_press(key, mod):
    global human_agent_action, human_wants_restart, human_sets_pause
    if key==0xff0d: human_wants_restart = True
    if key==32: human_sets_pause = not human_sets_pause
    a = int( key - ord('0') )
    if a <= 0 or a >= ACTIONS: return
    human_agent_action = a

def key_release(key, mod):
    global human_agent_action
    a = int( key - ord('0') )
    if a <= 0 or a >= ACTIONS: return
    if human_agent_action == a:
        human_agent_action = 0
env.unwrapped.viewer.window.on_key_press = key_press
env.unwrapped.viewer.window.on_key_release = key_release
        
def some_random_games_first():
    global human_agent_action
    for episode in range(10):
        env.reset()
        for t in range(200):
            env.render()
            action = human_agent_action
            observation, reward, done, info = env.step(action)
            
            if done:
                break;

AttributeError: 'NoneType' object has no attribute 'window'

In [187]:
some_random_games_first()

In [3]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score += reward
            if done:
                break
        if score >= score_req:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                    
                training_data.append([data[0], output])
        env.reset()
        scores.append(score)
    traning_data_save = np.array(training_data)
    np.save('saved.npy', traning_data_save)
    print('Average accepted score:',mean(accepted_scores))
    print('Median score for accepted scores:',median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data            

In [4]:
def neural_network_model(input_size):

    network = input_data(shape=[None, input_size, 1], name='input')

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')

    return model


def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]

    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input': X}, {'targets': y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_learning')
    return model

In [5]:
training_data = initial_population()

Average accepted score: 109.53333333333333
Median score for accepted scores: 106.0
Counter({102.0: 5, 101.0: 3, 106.0: 3, 100.0: 3, 132.0: 2, 107.0: 2, 110.0: 2, 111.0: 2, 115.0: 1, 105.0: 1, 104.0: 1, 112.0: 1, 117.0: 1, 113.0: 1, 114.0: 1, 155.0: 1})


In [6]:
model = train_model(training_data)

Training Step: 254  | total loss: 0.64661 | time: 0.497s
| Adam | epoch: 005 | loss: 0.64661 - acc: 0.6358 -- iter: 3200/3256
Training Step: 255  | total loss: 0.65067 | time: 0.508s
| Adam | epoch: 005 | loss: 0.65067 - acc: 0.6269 -- iter: 3256/3256
--


In [7]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        env.render()

        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0])

        choices.append(action)
                
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score+=reward
        if done: break

    scores.append(score)

print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
print(score_req)

Average Score: 188.7
choice 1:0.4854266030736619  choice 0:0.5145733969263381
100


In [174]:
for _ in range(100):
    print(env.step(random.randrange(0, 2)))

(array([-0.0440395 , -0.208931  ,  0.01553179,  0.32648553]), 1.0, False, {})
(array([-0.04821812, -0.01403358,  0.0220615 ,  0.03874084]), 1.0, False, {})
(array([-0.04849879,  0.18076517,  0.02283632, -0.24690064]), 1.0, False, {})
(array([-0.04488349,  0.37555366,  0.01789831, -0.53229389]), 1.0, False, {})
(array([-0.03737241,  0.57041935,  0.00725243, -0.8192838 ]), 1.0, False, {})
(array([-0.02596403,  0.7654413 , -0.00913325, -1.10967682]), 1.0, False, {})
(array([-0.0106552 ,  0.96068206, -0.03132678, -1.40521089]), 1.0, False, {})
(array([ 0.00855844,  1.15617862, -0.059431  , -1.70752042]), 1.0, False, {})
(array([ 0.03168201,  0.96178822, -0.09358141, -1.43391204]), 1.0, False, {})
(array([ 0.05091778,  0.76793695, -0.12225965, -1.17188024]), 1.0, False, {})
(array([ 0.06627652,  0.96441764, -0.14569726, -1.50025621]), 1.0, False, {})
(array([ 0.08556487,  1.1609777 , -0.17570238, -1.83465606]), 1.0, False, {})
(array([ 0.10878442,  1.35755443, -0.2123955 , -2.17637254]), 1.

In [173]:
env.reset()

array([-0.04376754, -0.01359789,  0.01494932,  0.02912367])

In [176]:
eval(input(''))

1


1